In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.utils import to_categorical
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from random import randint

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#sinky renamed to all_data
all_data = pd.read_csv('/content/drive/MyDrive/metrics_iclr_2022/titanic_survivors.csv')
all_data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [ ]:
def class_level_stats(ground_truth, predictions):
  conf_matrix = confusion_matrix(ground_truth, predictions)

  FP = conf_matrix.sum(axis=0) - np.diag(conf_matrix)
  FN = conf_matrix.sum(axis=1) - np.diag(conf_matrix)
  TP = np.diag(conf_matrix)
  TN = conf_matrix.sum() - (FP + FN + TP)

  FPR = FP / (FP  + TN)
  FNR = FN / (FN + TP)
  return FPR, FNR

In [ ]:
test = np.array(all_data['Survived'])

In [ ]:
train = np.zeros((891, 6), dtype=float)
embark_dic = {'S':0, 'C':1, 'Q':2, 'X':3}

embarks = ['X' if i != 'C' and i != 'S' and i != 'Q' else i for i in all_data['Embarked']]

print(all_data['Pclass'][0])

for i in range(len(all_data)):
  train[i][0] = all_data['Pclass'][i] #Passenger Class
  train[i][1] = all_data['Age'][i] #Passenger Age
  train[i][2] = all_data['SibSp'][i] #Number of siblings and spouses
  train[i][3] = all_data['Parch'][i] #Number of parents and children
  train[i][4] = all_data['Fare'][i] #Ammount Paid for the ticket
  train[i][5] =  embark_dic[embarks[i]]  #embarkation city

for i in range(len(train)):
  for j in range(len(train[0])):
    if np.isnan(train[i][j]):
      train[i][j] = 0                   
train


3


array([[ 3.    , 22.    ,  1.    ,  0.    ,  7.25  ,  0.    ],
       [ 1.    , 38.    ,  1.    ,  0.    , 71.2833,  1.    ],
       [ 3.    , 26.    ,  0.    ,  0.    ,  7.925 ,  0.    ],
       ...,
       [ 3.    ,  0.    ,  1.    ,  2.    , 23.45  ,  0.    ],
       [ 1.    , 26.    ,  0.    ,  0.    , 30.    ,  1.    ],
       [ 3.    , 32.    ,  0.    ,  0.    ,  7.75  ,  2.    ]])

In [ ]:
#0 indicates male
passenger_sex = np.array([0 if i=='male' else 1 for i in all_data['Sex']])

In [ ]:
X_train = train[0:700]  
X_test = train[700:891]
y_train = test[0:700] 
y_test = test[700:891]
passenger_sex_test = passenger_sex[700:891]

In [ ]:
#happy_boy renamed to shallow_nn
shallow_nn = Sequential([
                        Input(shape=X_train[0].shape),
                        Dense(25, activation='sigmoid'),
                        Dense(25, activation='sigmoid'),
                        Dense(2, activation='softmax')
])
shallow_nn.compile(optimizer='adam', loss='mse', metrics=['acc'])
shallow_nn.summary()


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 25)                175       
_________________________________________________________________
dense_10 (Dense)             (None, 25)                650       
_________________________________________________________________
dense_11 (Dense)             (None, 2)                 52        
Total params: 877
Trainable params: 877
Non-trainable params: 0
_________________________________________________________________


In [ ]:
y_train = to_categorical(y_train)
shallow_nn.fit(X_train, y_train, epochs=50, verbose=0)


In [ ]:
y_pred = np.argmax(shallow_nn.predict(X_test), axis=-1)
assert len(y_test)==len(y_pred), "wrong numbers of labels in prediction"


In [ ]:
all_FPR, all_FNR = class_level_stats(y_test, y_pred)
print('Women in test data: ', len(y_test[np.where(passenger_sex_test==1)]))
women_FPR, women_FNR = class_level_stats(y_test[np.where(passenger_sex_test==1)], y_pred[np.where(passenger_sex_test==1)])
men_FPR, men_FNR = class_level_stats(y_test[np.where(passenger_sex_test==0)], y_pred[np.where(passenger_sex_test==0)])
# print(y_pred[np.where(passenger_sex_test==1)])
# print(y_test[np.where(passenger_sex_test==1)])
print('NN All FPR: ', all_FPR)
print('NN All FNR: ', all_FNR)
print('NN Women FPR: ', women_FPR)
print('NN Women FNR: ', women_FNR)
print('NN Men FPR: ', men_FPR)
print('NN Men FNR: ', men_FNR)
print('----------All Data CM--------------')
print(confusion_matrix( y_test, y_pred))
print('----------Women CM--------------')
print(confusion_matrix(y_test[np.where(passenger_sex_test==1)], y_pred[np.where(passenger_sex_test==1)]))
print('----------Men CM--------------')
print(confusion_matrix(y_test[np.where(passenger_sex_test==0)], y_pred[np.where(passenger_sex_test==0)]))


Women in test data:  63
NN All FPR:  [0.47887324 0.15833333]
NN All FNR:  [0.15833333 0.47887324]
NN Women FPR:  [0.42553191 0.1875    ]
NN Women FNR:  [0.1875     0.42553191]
NN Men FPR:  [0.58333333 0.15384615]
NN Men FNR:  [0.15384615 0.58333333]
----------All Data CM--------------
[[101  19]
 [ 34  37]]
----------Women CM--------------
[[13  3]
 [20 27]]
----------Men CM--------------
[[88 16]
 [14 10]]


In [ ]:
#generate random predictions for normalization
y_rand = [randint(0,1) for i in range(len(y_test))]
rand_FPR, rand_FNR = class_level_stats(y_test, y_rand)
print('Random FPR: ', rand_FPR)
print('Random FNR: ', rand_FNR)

Random FPR:  [0.45070423 0.49166667]
Random FNR:  [0.49166667 0.45070423]


In [ ]:
def my_special_variance(error_rates):
  assert error_rates.ndim == 2, 'Error rates must be 2 dimensional'
  my_special_mean = error_rates.sum(axis=0)/[len(error_rates), len(error_rates)]
  sum_of_squares = 0
  for e in error_rates:
    sum_of_squares += ((my_special_mean[0]-e[0])**2 + (my_special_mean[1]-e[1])**2)
  return sum_of_squares/len(error_rates)

In [1]:
def dis_from_sym(p):
    return np.mean(np.abs(p[:, 0] - p[:, 1]))

def return_compression_errors(base_fpr, base_fnr, fpr, fnr):
    
    classes = 1000
    
    # get method stats
    FPR_change = (fpr - base_fpr)/base_fpr 
    FNR_change = (fnr - base_fnr)/base_fnr 
    
    # make class points
    points = np.dstack((FPR_change, FNR_change))[0]
    
    #we should use my hand-coded variance, or mse
    return my_special_variance(points), dis_from_sym(points)

In [ ]:
women_CEV, women_SDE = return_compression_errors(all_FPR, all_FNR, women_FPR, women_FNR)
men_CEV, men_SDE = return_compression_errors(all_FPR, all_FNR, men_FPR, men_FNR)
print('NN Women CEV: ', women_CEV)
print('NN Women SDE: ', women_SDE)
print('NN Men CEV: ', men_CEV)
print('NN Men SDE: ', men_SDE)

NN Women CEV:  0.04368960990187911
NN Women SDE:  0.29559976286147155
NN Men CEV:  0.030375538549678827
NN Men SDE:  0.24647733587362075


In [ ]:
#rand CEV and SDE to normalize values
rand_CEV, rand_SDE = return_compression_errors(all_FPR, all_FNR, rand_FPR, rand_FNR)
print('Random CEV: ', rand_CEV)
print('Random SDE: ', rand_SDE)
#print normalized NN values
print('NN Normalized Women CEV: ', women_CEV/rand_CEV)
print('NN Normalized Women SDE: ', women_SDE/rand_SDE)
print('NN Normalized Men CEV: ', men_CEV/rand_CEV)
print('NN Normalized Men SDE: ', men_SDE/rand_SDE)

Random CEV:  2.3416355950886136
Random SDE:  2.1640866873065012
NN Normalized Women CEV:  0.018657732225079958
NN Normalized Women SDE:  0.13659330959120933
NN Normalized Men CEV:  0.012971932359325677
NN Normalized Men SDE:  0.11389439125490632


In [ ]:
#Let's check the other guys
#Measuring and Mitigating Unintended Bias in Text Classification

cm_all = confusion_matrix(y_test, y_pred)
cm_wom = confusion_matrix(y_test[np.where(passenger_sex_test==1)], y_pred[np.where(passenger_sex_test==1)])
cm_men = confusion_matrix(y_test[np.where(passenger_sex_test==0)], y_pred[np.where(passenger_sex_test==0)])

fp_all = cm_all[0][1] / (cm_all[0][1] + cm_all[1][1])
fn_all = cm_all[1][0] / (cm_all[1][0] + cm_all[0][0])

fp_wom = cm_wom[0][1] / (cm_wom[0][1] + cm_wom[1][1])
fn_wom = cm_wom[1][0] / (cm_wom[1][0] + cm_wom[0][0])

fp_men = cm_men[0][1] / (cm_men[0][1] + cm_men[1][1])
fn_men = cm_men[1][0] / (cm_men[1][0] + cm_men[0][0])

fped = abs(fp_all-fp_wom)+abs(fp_all-fp_men)
fned = abs(fn_all-fn_wom)+abs(fn_all-fn_men)
print('NN False Positive Equality Differnce: ', fped)
print('NN False Negative Equality Differnce: ', fned)

#Evaluating Fairness Metrics in the Presence of Dataset Bias
y_pred_men = y_pred[np.where(passenger_sex_test==0)]
y_pred_wom = y_pred[np.where(passenger_sex_test==1)]
y_true_men = y_test[np.where(passenger_sex_test==0)]
y_true_wom = y_test[np.where(passenger_sex_test==1)]
dims = sum(y_pred_men)/len(y_pred_men) - sum(y_pred_wom)/len(y_pred_wom)
diamr = sum(y_pred_men-y_true_men)/len(y_pred_men) - \
  sum(y_pred_wom-y_true_wom)/len(y_pred_wom)

print('NN Diffference in mean scores: ', dims)
print('NN Difference in average model residuals: ', diamr)

NN False Positive Equality Differnce:  0.5153846153846154
NN False Negative Equality Differnce:  0.46880570409982175
NN Diffference in mean scores:  -0.27306547619047616
NN Difference in average model residuals:  0.2854662698412698


In [ ]:



cols = pd.MultiIndex.from_tuples([
          ("CEV", "All->Men"), 
          ("CEV", "All->Women"), 
          ("SDE", "All->Men"), 
          ("SDE", "All->Women"), 
          ('Error Rate Equality Difference', 'FPED'),
          ('Error Rate Equality Difference', 'FNED'),
          ('Difference in Expected Values', 'DIMS'),
          ('Difference in Expected Values', 'DIAMR'),
])
df = pd.DataFrame([[
              men_CEV, women_CEV, men_SDE, women_SDE,
              fped, fned, dims, diamr]]
    , columns=cols)
df

CEV             ... Difference in Expected Values          
   All->Men All->Women  ...                          DIMS     DIAMR
0  0.030376    0.04369  ...                     -0.273065  0.285466

[1 rows x 8 columns]

In [ ]:
df.to_latex()

'\\begin{tabular}{lrrrrrrrr}\n\\toprule\n{} & \\multicolumn{2}{l}{CEV} & \\multicolumn{2}{l}{SDE} & \\multicolumn{2}{l}{Error Rate Equality Difference} & \\multicolumn{2}{l}{Difference in Expected Values} \\\\\n{} &  All->Men & All->Women &  All->Men & All->Women &                           FPED &      FNED &                          DIMS &     DIAMR \\\\\n\\midrule\n0 &  0.030376 &    0.04369 &  0.246477 &     0.2956 &                       0.515385 &  0.468806 &                     -0.273065 &  0.285466 \\\\\n\\bottomrule\n\\end{tabular}\n'

In [ ]:
df.to_csv()

',CEV,CEV,SDE,SDE,Error Rate Equality Difference,Error Rate Equality Difference,Difference in Expected Values,Difference in Expected Values\n,All->Men,All->Women,All->Men,All->Women,FPED,FNED,DIMS,DIAMR\n0,0.030375538549678827,0.04368960990187911,0.24647733587362075,0.29559976286147155,0.5153846153846154,0.46880570409982175,-0.27306547619047616,0.2854662698412698\n'

In [ ]:
#Build and train SVM
#sad_boy renamed to svm_class
if y_train.ndim > 1:
  y_train = np.argmax(y_train, axis=-1)
svm_class = SVC(gamma='auto')
svm_class.fit(X_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [ ]:
#Overwrite y_pred with new predictions
y_pred = svm_class.predict(X_test)

In [ ]:
#Overwrite previous FPRs and FNRs
all_FPR, all_FNR = class_level_stats(y_test, y_pred)
print('Women in test data: ', len(y_test[np.where(passenger_sex_test==1)]))
women_FPR, women_FNR = class_level_stats(y_test[np.where(passenger_sex_test==1)], y_pred[np.where(passenger_sex_test==1)])
men_FPR, men_FNR = class_level_stats(y_test[np.where(passenger_sex_test==0)], y_pred[np.where(passenger_sex_test==0)])
print('SVM All FPR: ', all_FPR)
print('SVM All FNR: ', all_FNR)
print('SVM Women FPR: ', women_FPR)
print('SVM Women FNR: ', women_FNR)
print('SVM Men FPR: ', men_FPR)
print('SVM Men FNR: ', men_FNR)
print('----------All Data CM--------------')
print(confusion_matrix( y_test, y_pred))
print('----------Women CM--------------')
print(confusion_matrix(y_test[np.where(passenger_sex_test==1)], y_pred[np.where(passenger_sex_test==1)]))
print('----------Men CM--------------')
print(confusion_matrix(y_test[np.where(passenger_sex_test==0)], y_pred[np.where(passenger_sex_test==0)]))

Women in test data:  63
SVM All FPR:  [0.56338028 0.175     ]
SVM All FNR:  [0.175      0.56338028]
SVM Women FPR:  [0.63829787 0.1875    ]
SVM Women FNR:  [0.1875     0.63829787]
SVM Men FPR:  [0.41666667 0.17307692]
SVM Men FNR:  [0.17307692 0.41666667]
----------All Data CM--------------
[[99 21]
 [40 31]]
----------Women CM--------------
[[13  3]
 [30 17]]
----------Men CM--------------
[[86 18]
 [10 14]]


In [ ]:
#Overwrite previous CEV and SDE scores
women_CEV, women_SDE = return_compression_errors(all_FPR, all_FNR, women_FPR, women_FNR)
men_CEV, men_SDE = return_compression_errors(all_FPR, all_FNR, men_FPR, men_FNR)
print('SVM Women CEV: ', women_CEV)
print('SVM Women SDE: ', women_SDE)
print('SVM Men CEV: ', men_CEV)
print('SVM Men SDE: ', men_SDE)

SVM Women CEV:  0.0018942106041148876
SVM Women SDE:  0.06155015197568382
SVM Men CEV:  0.031107077708425593
SVM Men SDE:  0.24942765567765574


In [ ]:
#print normalized NN values
print('SVM Normalized Women CEV: ', women_CEV/rand_CEV)
print('SVM Normalized Women SDE: ', women_SDE/rand_SDE)
print('SVM Normalized Men CEV: ', men_CEV/rand_CEV)
print('SVM Normalized Men SDE: ', men_SDE/rand_SDE)

SVM Normalized Women CEV:  0.0008089262941201599
SVM Normalized Women SDE:  0.02844162959677522
SVM Normalized Men CEV:  0.013284337568864305
SVM Normalized Men SDE:  0.1152577006922501


In [ ]:
#Overwrite previous other metrics
cm_all = confusion_matrix(y_test, y_pred)
cm_wom = confusion_matrix(y_test[np.where(passenger_sex_test==1)], y_pred[np.where(passenger_sex_test==1)])
cm_men = confusion_matrix(y_test[np.where(passenger_sex_test==0)], y_pred[np.where(passenger_sex_test==0)])

fp_all = cm_all[0][1] / (cm_all[0][1] + cm_all[1][1])
fn_all = cm_all[1][0] / (cm_all[1][0] + cm_all[0][0])

fp_wom = cm_wom[0][1] / (cm_wom[0][1] + cm_wom[1][1])
fn_wom = cm_wom[1][0] / (cm_wom[1][0] + cm_wom[0][0])

fp_men = cm_men[0][1] / (cm_men[0][1] + cm_men[1][1])
fn_men = cm_men[1][0] / (cm_men[1][0] + cm_men[0][0])

fped = abs(fp_all-fp_wom)+abs(fp_all-fp_men)
fned = abs(fn_all-fn_wom)+abs(fn_all-fn_men)
print('SVM False Positive Equality Differnce: ', fped)
print('SVM False Negative Equality Differnce: ', fned)

#Evaluating Fairness Metrics in the Presence of Dataset Bias
y_pred_men = y_pred[np.where(passenger_sex_test==0)]
y_pred_wom = y_pred[np.where(passenger_sex_test==1)]
y_true_men = y_test[np.where(passenger_sex_test==0)]
y_true_wom = y_test[np.where(passenger_sex_test==1)]
dims = sum(y_pred_men)/len(y_pred_men) - sum(y_pred_wom)/len(y_pred_wom)
diamr = sum(y_pred_men-y_true_men)/len(y_pred_men) - \
  sum(y_pred_wom-y_true_wom)/len(y_pred_wom)

print('SVM Diffference in mean scores: ', dims)
print('SVM Difference in average model residuals: ', diamr)

SVM False Positive Equality Differnce:  0.4125
SVM False Negative Equality Differnce:  0.5935077519379846
SVM Diffference in mean scores:  -0.06746031746031744
SVM Difference in average model residuals:  0.49107142857142855


In [ ]:
#Build and train Gradient Tree Boosting Classifier
#Build and train SVM
#hungry_boy renamed to GTB_class
if y_train.ndim > 1:
  y_train = np.argmax(y_train, axis=-1)
GTB_class = GradientBoostingClassifier(loss='exponential').fit(X_train, y_train)
GTB_class.fit(X_train, y_train) 

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='exponential', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [ ]:
#Overwrite y_pred with new predictions
y_pred = GTB_class.predict(X_test)

In [ ]:
#Overwrite previous FPRs and FNRs
all_FPR, all_FNR = class_level_stats(y_test, y_pred)
print('Women in test data: ', len(y_test[np.where(passenger_sex_test==1)]))
women_FPR, women_FNR = class_level_stats(y_test[np.where(passenger_sex_test==1)], y_pred[np.where(passenger_sex_test==1)])
men_FPR, men_FNR = class_level_stats(y_test[np.where(passenger_sex_test==0)], y_pred[np.where(passenger_sex_test==0)])
print('GTB All FPR: ', all_FPR)
print('GTB All FNR: ', all_FNR)
print('GTB Women FPR: ', women_FPR)
print('GTB Women FNR: ', women_FNR)
print('GTB Men FPR: ', men_FPR)
print('GTB Men FNR: ', men_FNR)
print('----------All Data CM--------------')
print(confusion_matrix( y_test, y_pred))
print('----------Women CM--------------')
print(confusion_matrix(y_test[np.where(passenger_sex_test==1)], y_pred[np.where(passenger_sex_test==1)]))
print('----------Men CM--------------')
print(confusion_matrix(y_test[np.where(passenger_sex_test==0)], y_pred[np.where(passenger_sex_test==0)]))

Women in test data:  63
GTB All FPR:  [0.50704225 0.13333333]
GTB All FNR:  [0.13333333 0.50704225]
GTB Women FPR:  [0.5106383 0.125    ]
GTB Women FNR:  [0.125     0.5106383]
GTB Men FPR:  [0.5        0.13461538]
GTB Men FNR:  [0.13461538 0.5       ]
----------All Data CM--------------
[[104  16]
 [ 36  35]]
----------Women CM--------------
[[14  2]
 [24 23]]
----------Men CM--------------
[[90 14]
 [12 12]]


In [ ]:
#Overwrite previous CEV and SDE scores
women_CEV, women_SDE = return_compression_errors(all_FPR, all_FNR, women_FPR, women_FNR)
men_CEV, men_SDE = return_compression_errors(all_FPR, all_FNR, men_FPR, men_FNR)
print('GTB Women CEV: ', women_CEV)
print('GTB Women SDE: ', women_SDE)
print('GTB Men CEV: ', men_CEV)
print('GTB Men SDE: ', men_SDE)

GTB Women CEV:  0.0024215370517076556
GTB Women SDE:  0.06959219858156021
GTB Men CEV:  0.0002762254364818455
GTB Men SDE:  0.023504273504273452


In [ ]:
#print normalized NN values
print('GTB Normalized Women CEV: ', women_CEV/rand_CEV)
print('GTB Normalized Women SDE: ', women_SDE/rand_SDE)
print('GTB Normalized Men CEV: ', men_CEV/rand_CEV)
print('GTB Normalized Men SDE: ', men_SDE/rand_SDE)

GTB Normalized Women CEV:  0.001034122071250808
GTB Normalized Women SDE:  0.032157768443267454
GTB Normalized Men CEV:  0.00011796260573643716
GTB Normalized Men SDE:  0.010861059144320924


In [ ]:
#Overwrite previous other metrics
cm_all = confusion_matrix(y_test, y_pred)
cm_wom = confusion_matrix(y_test[np.where(passenger_sex_test==1)], y_pred[np.where(passenger_sex_test==1)])
cm_men = confusion_matrix(y_test[np.where(passenger_sex_test==0)], y_pred[np.where(passenger_sex_test==0)])

fp_all = cm_all[0][1] / (cm_all[0][1] + cm_all[1][1])
fn_all = cm_all[1][0] / (cm_all[1][0] + cm_all[0][0])

fp_wom = cm_wom[0][1] / (cm_wom[0][1] + cm_wom[1][1])
fn_wom = cm_wom[1][0] / (cm_wom[1][0] + cm_wom[0][0])

fp_men = cm_men[0][1] / (cm_men[0][1] + cm_men[1][1])
fn_men = cm_men[1][0] / (cm_men[1][0] + cm_men[0][0])

fped = abs(fp_all-fp_wom)+abs(fp_all-fp_men)
fned = abs(fn_all-fn_wom)+abs(fn_all-fn_men)
print('GTB False Positive Equality Differnce: ', fped)
print('GTB False Negative Equality Differnce: ', fned)

#Evaluating Fairness Metrics in the Presence of Dataset Bias
y_pred_men = y_pred[np.where(passenger_sex_test==0)]
y_pred_wom = y_pred[np.where(passenger_sex_test==1)]
y_true_men = y_test[np.where(passenger_sex_test==0)]
y_true_wom = y_test[np.where(passenger_sex_test==1)]
dims = sum(y_pred_men)/len(y_pred_men) - sum(y_pred_wom)/len(y_pred_wom)
diamr = sum(y_pred_men-y_true_men)/len(y_pred_men) - \
  sum(y_pred_wom-y_true_wom)/len(y_pred_wom)

print('GTB Diffference in mean scores: ', dims)
print('GTB Difference in average model residuals: ', diamr)

GTB False Positive Equality Differnce:  0.4584615384615384
GTB False Negative Equality Differnce:  0.5139318885448916
GTB Diffference in mean scores:  -0.1937003968253968
GTB Difference in average model residuals:  0.3648313492063492
